<a href="https://colab.research.google.com/github/BrianKEverett/County-Line/blob/main/Dissertation4_Everett.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install ydata-profiling
from ydata_profiling import ProfileReport

In [16]:
import time, os, sys, re
import zipfile, json, datetime, string
import numpy as np
from statistics import *

import matplotlib.pyplot as plt

import pandas as pd
import pandas_datareader as pdr
from pandas_datareader import wb
from pandas.io.formats.style import Styler
import plotly.express as px

import missingno as msno

from google.colab import files

import seaborn as sns

from google.colab import data_table
data_table.enable_dataframe_formatter()
data_table.max_columns = 50

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

plt.style.use('classic')

In [17]:
permits=pd.read_csv("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Permits.csv")
# dataset can be found here: https://njdca.maps.arcgis.com/home/item.html?id=c754e8f800424bcbb6ad4e6e85b9f736 from NJ Dept of Community Affairs Website.
#this dataset was chosen mostly to assist with my dissertation proposal - I am seeking to explore the behaviors of planning boards in New Jersey, and one hypothesis I have is that planning board decisions are influenced by the effects of the County Line Balloting system, which is unique to 19 out of 21 counties in New Jersey. No other state in the US runs primary elections in this way.
#More on the County Line can be found here by Julia Sass Rubin: https://www.njpp.org/wp-content/uploads/2021/01/NJPP-Report-Does-the-County-Line-Matter-Update-wiht-Final-Vote-Counts.pdf

taxes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/mediantax.csv")
# dataset can be retireved via: https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b from NJ Department of Community Affairs
# Another hypothesis I have is that the public narrative put forth by planning board members when approving controversial permits, "this will increase rateables for the township, lowering your taxes", does not actually come to fruition
#This data set is helpful for exploring that narrative, and seeing if the opposite is occurring, i.e. more development actually yields higher property taxes

municodes=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Municodes.csv")
#Data file of all NJ municiaplities, and counties, with the corresponding municipalitiy DCA code. This data file will be most helpful for matching and merging.

#crime=pd.read_csv ("https://raw.githubusercontent.com/BrianKEverett/County-Line/main/CamdenCrime.csv") #not a good format for reading data!
#Dataset can be retireved here: https://www.nj.gov/njsp/ucr/uniform-crime-reports.shtml on the NJ Office of the Attorney General's website
#This crime data is important to consider when analyzing planning and zoning. Does any specific type of development correlate with increased crime? Can liveability theory be worked in here for whether or not communities have what they need to prevent crime?

jobs=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/jobsdensity.csv')
#dataset can be built via the table selections on the NJ Community Affairs website - https://njdca.maps.arcgis.com/apps/webappviewer/index.html?id=96ec274c50a34890b23263f101e4ad9b
#Job density is a good variable to consider regarding new large dollar permits. Are some places growing more than others? Can this be attribute to the phenomenon of the County Line?

countysize=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/NJCountySize.csv')
#dataset can be found via Wikipedia via 2020 census data - https://en.wikipedia.org/wiki/List_of_counties_in_New_Jersey

njtowns=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/njtowns2.csv')
#Dataset can be found at: https://en.wikipedia.org/wiki/List_of_municipalities_in_New_Jersey#:~:text=The%20largest%20municipality%20by%20population,most%20populous%20being%20South%20Carolina.

njcodes=pd.read_csv('https://raw.githubusercontent.com/BrianKEverett/County-Line/main/Municipalities_of_New_Jersey_20240131.csv')
#data was retireved via State of New Jersey Open Data Center: https://data.nj.gov/Reference-Data/Municipalities-of-New-Jersey/k9xb-zgh4/data_preview

In [18]:
permits = permits.rename(columns={'DCA MUNI CODE': 'DCA'})
permits = permits.rename(columns={'MUNICIPALITY': 'Municipality'})
permits["Municipality"]= permits["Municipality"].str.title()
permits['Municipality'] = permits['Municipality'].str.replace(' Boro', ' Borough')
permits['Municipality'] = permits['Municipality'].str.replace(' Twp', ' Township')
del permits['ID']
del permits['BLOCK NUMBER']
del permits['PAMS PIN']
del permits['USE GROUP']
del permits['YCOORD']
del permits['XCOORD']
del permits['MATCH TYPE']
del permits['LOT NUMBER']
del permits['DATE ISSUED']
del permits['TAX CODE']
permits = permits.rename(columns={'TYPE': 'Permits'})

del municodes['MUNICIPALITY_CODE_DCA']
del municodes['MUNICIPALITY_NAME_DCA']
del municodes['MUNICIPALITY_CODE_GNIS']
del municodes['MUNICIPALITY_NAME_GNIS']
del municodes['MUNICIPALITY_CODE_FIPS']
municodes = municodes.rename(columns={'MUNICIPALITY_NAME_NJ-1040': 'Municipality'})
municodes = municodes.rename(columns={'MUNICIPALITY_CODE_NJ-1040': 'DCA'})
municodes = municodes.rename(columns={'COUNTY_NAME_COMMON': 'County'})
del municodes['MUNICIPALITY_NAME_COMMON']
municodes['County'] = municodes['County'].str.replace(' County', '')
municodes = municodes.set_index('Municipality')

countysize['Largest City Population']=countysize['Largest City Population'].str.replace(',','')

njtowns = njtowns.rename(columns={'Name': 'Municipality'})
del njtowns['Type']
del njtowns['Unnamed: 6']
del njtowns['Unnamed: 7']
del njtowns['Unnamed: 8']
del njtowns['Unnamed: 9']
del njtowns['Unnamed: 10']
njtowns.replace(',','', regex=True, inplace=True)

#jobs = jobs.set_index('Municipality')
del jobs['JobsVintage']
del jobs['Blk_Grp_Name']
del jobs['JobsDensity']

#taxes = taxes.set_index('Municipality')
del taxes['Tract_Name']
del taxes['Data_Vintage']
del taxes[' ']

#njcodes
del njcodes ['MUNICIPALITY_NAME_COMMON']
del njcodes ['MUNICIPALITY_CODE_NJ-1040']
del njcodes ['MUNICIPALITY_NAME_NJ-1040']
del njcodes ['MUNICIPALITY_NAME_GNIS']
njcodes = njcodes.rename(columns={'COUNTY_NAME_COMMON': 'County'})
njcodes = njcodes.rename(columns={'MUNICIPALITY_NAME_DCA': 'Municipality'})
njcodes = njcodes.rename(columns={'MUNICIPALITY_CODE_DCA': 'DCA'})
njcodes = njcodes.rename(columns={'MUNICIPALITY_CODE_GNIS': 'GNIS'})
njcodes = njcodes.rename(columns={'MUNICIPALITY_CODE_FIPS': 'FIPS'})
njcodes['County'] = njcodes['County'].str.replace(' County', '')
njcodes['Municipality'] = njcodes['Municipality'].str.replace('Atlantic City City', 'Atlantic City')

In [19]:
njcodes

,County,Municipality,DCA,GNIS,FIPS
0,Atlantic,Absecon City,101,885134,3400100100
1,Atlantic,Atlantic City,102,885142,3400102080
2,Atlantic,Brigantine City,103,885171,3400107810
3,Atlantic,Buena Borough,104,885173,3400108680
4,Atlantic,Buena Vista Township,105,882048,3400108710
...,...,...,...,...,...
559,Warren,Phillipsburg Town,2119,885350,3404158350
560,Warren,Pohatcong Township,2120,882254,3404159820
561,Warren,Washington Borough,2121,885432,3404177270
562,Warren,Washington Township,2122,882250,3404177300
